In [1]:
import os, random, time
import matplotlib.pyplot as plt, matplotlib.ticker as ticker, numpy as np, scipy as sp, scipy.constants as ct, scipy.optimize as sciopt
%matplotlib tk
plt.rcParams.update({'font.size': 20})
from collections import defaultdict
from math import floor, ceil, log, pi, sqrt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline
from scipy.signal import savgol_filter, find_peaks, hann, hilbert
from scipy.optimize import minimize
from scipy.misc import derivative
from numpy.fft import rfft, rfftfreq, irfft

import PyLTSpice
from PyLTSpice.LTSpice_RawRead import LTSpiceRawRead
from PyLTSpice.LTSpiceBatch import LTCommander

import VoltageGeneration

Found Numpy. WIll be used for storing data


In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# Ideal Electrode Voltages

In [3]:
filepath = 'MB_transport_2us_c2c/'
exp = 'predistortion2'
N0 = 5
dt = 1e-6
buffer1 = 290
buffer2 = 0

basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filenames = sorted(os.listdir(''.join([basepath, filepath])), key = lambda x: float(x.split('_')[-1][:-4]))
filedata = [np.load(''.join([basepath, filepath]) + fname, allow_pickle = True).tolist() for fname in filenames[1:-1]]

Ntot = buffer1+N0+buffer2
T0 = dt*Ntot
# electrode_ordering = [
#     'S21', 'N1', 'S20', 'N2', 'S19', 'N3', 'S14',
#     'N4', 'S13', 'N8', 'S12', 'N9', 'S11', 'N10',
#     'S10', 'N11', 'S9', 'N12', 'S8', 'N13', 'S4',
#     'N14', 'S3', 'N19', 'S2', 'N20', 'S1', 'N21',
#     'S25', 'S24', 'S23', 'S22'
# ] # SLT
electrode_ordering = [
    'S21', 'N1', 'S20', 'N2', 'S19', 'N3', 'S14',
    'N4', 'S13', 'N8', 'S12', 'N9', 'S11', 'N10',
    'S10', 'N11', 'S9', 'N12', 'S8', 'N13', 'S4',
    'N14', 'S3', 'N19', 'S2', 'N20', 'S1', 'N21',
    'S25', 'S24', 'S23', 'S22'
] # MaxBeta
    
d = defaultdict(list)
for f in filedata:
    [d[k].append(v) for k, v in f.items()]

%matplotlib tk
v_ideal = {}
v_ideal_deconv = {}
voltages = {}
plt.figure('voltages')
for k, v in d.items():
    vevents = np.concatenate(([v[0]], v, [v[-1]]))
    tevents = np.concatenate(([0], dt*np.linspace(buffer1, N0+buffer1, len(v)), [T0]))
    
    ## TWEAK VOLTAGE SETS
#     vevents = savgol_filter(vevents, 19, 3) ## 19

    ## PLOT VOLTAGE SETS
    mark = '.'
    if k[0] == 'S':
        mark = 'x'
    plt.plot([t*1e6 for t in tevents], vevents, marker=mark, label=k)
    v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
    v_ideal_deconv[k] = interp1d(np.concatenate((tevents, 2*T0-tevents[::-1])), np.concatenate((vevents, vevents[::-1])))
    voltages[k] = v_ideal[k].__call__([(n+0.5)*dt for n in range(Ntot)])
#     voltages[k] = v_ideal_deconv[k].__call__([(n+0.5)*dt for n in range(Ntot*2)])

plt.legend(loc='best', prop={'size': 12})
plt.ylabel('Electrode Voltage (V)')
plt.xlabel('Time (us)')
# np.save(basepath + '/'.join((exp, 'STA_crosstalk_better.npy')), voltages)


Text(0.5, 0, 'Time (us)')

In [3]:
#### Bang-Bang on the MaxBeta
filepath = 'predistortion4/'
N0 = 5
dt = 0.392e-6
buffer1 = 1000
buffer2 = 0

basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filenames = {s:s+'_ideal_withbuffers.npy' for s in ['S13', 'N8', 'S12', 'N9']}
filedata = {k:np.load(''.join([basepath, filepath]) + v, allow_pickle = True).tolist() for k, v in filenames.items()}

Ntot = buffer1+N0+buffer2
T0 = dt*Ntot
    
d = defaultdict(list)
for k, v in filedata.items():
    d[k] = v

%matplotlib tk
v_ideal = {}
v_ideal_deconv = {}
v_DAC = {}
plt.figure('voltages')
for k, v in d.items():
    mark = '.'
    if k[0] == 'S':
        mark = 'x'
        
    vevents = np.concatenate(([v[0]], v, [v[-1]]))
    tevents = np.concatenate(([-0.1e-6], dt*np.linspace(buffer1, N0+buffer1, len(v)), [T0+0.1e-6]))
    
#     vevents = savgol_filter(vevents, 19, 3) ## 19
    
    plt.plot([t*1e6 for t in tevents], vevents, marker=mark, label=k)
    v_ideal[k] = interp1d(np.append(tevents, 1), np.append(vevents, vevents[-1]))
    v_ideal_deconv[k] = interp1d(np.concatenate((tevents, 2*T0-tevents[::-1])), np.concatenate((vevents, vevents[::-1])))
    v_DAC[k] = v_ideal[k].__call__([(n+0.5)*dt for n in range(Ntot)])
#     voltages[k] = v_ideal_deconv[k].__call__([(n+0.5)*dt for n in range(Ntot*2)])

plt.legend(loc='best', prop={'size': 12})
plt.ylabel('Electrode Voltage (V)')
plt.xlabel('Time (us)')


Text(0.5, 0, 'Time (us)')

# Transport Protocol Methods

In [4]:
def qtanhN(t, tspan, N=5, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if (t-t0) < T:
        return y0 + max((yf-y0)/2*(np.tanh(N*(2*(t-t0)-T)/T)+np.tanh(N))/np.tanh(N), 0)
    else:
        return yf
    
def qsin(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T = tf-t0
    if t < t0:
        return y0
    elif (t-t0) < T:
        return y0 + max((yf-y0)/2*(1-np.cos(pi*(t-t0)/T)), 0)
    else:
        return yf
    
def get_freq_from_alpha(a, m_=40*ct.u):
    ''' make sure alpha is in V/um^2'''
    return np.sign(a)*(2*np.abs(a)*1e12*ct.e/m_)**0.5/(2*pi)

def get_alpha_from_freq(f, m_=40*ct.u):
    return (2*pi*f)**2/(2*ct.e/m_)

def qsta(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(60*s - 180*s**2 + 120*s**3) + 10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def qsta2(t, tspan, qspan=(0, 120), f=1):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*((1/(2*pi*f)**2)*(2520*s**3-12600*s**4+22680*s**5-17640*s**6+5040*s**7)
                             + 126*s**5-420*s**6+540*s**7-315*s**8+70*s**9)
    else:
        return yf
    
def qtrig(t, tspan, qspan=(0, 120), f=1.2e6, mu=1):
    ''' args should be f (axialfreq in Hz), mu (mass ratio)'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti, d = tf-t0, t-t0, yf-y0
    s = ti/T
    Op, On = 2*np.pi*f*(1+1/mu+(1-1/mu+1/mu**2)**0.5)**0.5, 2*np.pi*f*(1+1/mu-(1-1/mu+1/mu**2)**0.5)**0.5
    b3 = -49*((T*Op)**2 - (5*np.pi)**2)*((T*On)**2 - (5*np.pi)**2)/(2048*(T*T*On*Op)**2)
    b4 = 5*((T*Op)**2 - (7*np.pi)**2)*((T*On)**2 - (7*np.pi)**2)/(2048*(T*T*On*Op)**2)
    
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + d*(0.5 + (-9/16+2*b3+5*b4)*np.cos(np.pi*s) + 1/16*(1-48*b3-96*b4)*np.cos(3*np.pi*s) + b3*np.cos(5*np.pi*s) + b4*np.cos(7*np.pi*s))
    else:
        return yf
    
def d_t(t, tspan, qspan=(0, 120)):
    ''' t is a specific time
        tspan is a tuple with the start and end time'''
    t0, tf = tspan
    y0, yf = qspan
    T, ti = tf-t0, t-t0 
    s = ti/T
    if s < 0:
        return y0
    elif s <= 1:
        return y0 + (yf-y0)*(10*s**3 - 15*s**4 + 6*s**5)
    else:
        return yf
    
def gaussian(x, mu, sig):
    return np.exp(-(x-mu)*(x-mu)/(2*sig*sig))

In [5]:
f0 = 4e6 # Axial frequency [Hz]
T = 1.96e-6 # Transport time [s]
move = (186.5, 231.5) # Start and End positions [um]
amu = ct.m_u
Ca = 40*amu
a0 = get_alpha_from_freq(f0, m_=Ca)*1e-12 # Specify ion mass. [Units are V/m^2, but ITVG uses um as the length scale]

t_ideal = np.linspace(0, T, 100)
q_ideal = np.array([qsta2(t, (0, T), qspan=move, f=T*f0) for t in t_ideal])
# q_ideal = np.linspace(move[0], move[1], len(t_ideal))
a_ideal = np.repeat(a0, len(t_ideal))
q_ideal_func = interp1d(np.append(t_ideal, 1), np.append(q_ideal, q_ideal[-1]))
a_ideal_func = interp1d(np.append(t_ideal, 1), np.append(a_ideal, a_ideal[-1]))

plt.plot(t_ideal*1e6, q_ideal)
plt.xlabel('time (us)')
plt.ylabel('position (um)')

Text(0, 0.5, 'position (um)')

# PyLTSpice Initialization

In [5]:
def set_vinput(volt_array, dt, st, sr):
    ''' writes PWL input to 'input.txt' '''    
    N = len(volt_array)
    varray = [v/16+2.5 for v in volt_array] ## fastino
#     varray = volt_array ## zotino
    
    tevents = [0]
    vevents = [varray[0]]
    for i in range(1,N):
        ti = dt*i
        vi = vevents[-1]
        tevents.append(ti)
        vevents.append(vi)
        
        tdelay = abs(varray[i] - vi)/sr
#         tdelay = 1e-12
        if tdelay > dt:
            print('WARNING: SLEW RATE LONGER THAN TIME STEP')
            tevents.append(ti + dt)
            vevents.append(vi + (varray[i] - vi)*dt/tdelay)
        else:
            tevents.append(ti + tdelay)
            vevents.append(varray[i])
    tevents.append(N*dt)
    vevents.append(varray[-1])
    
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip(tevents, vevents)]

    return tevents, vevents

def run_spice_sim(endsim, st, attempts=1):
    instruction = '.TRAN' + ' '.join((str(st), str(endsim)))
    LTC.netlist.insert(-2, instruction)
    LTC.write_netlist()
    rawfile, logfile = LTC.run()
    LTC.reset_netlist()

    if rawfile:
        for i in range(attempts):
            try:
                LTR = LTSpiceRawRead(rawfile)
                break
            except UnicodeDecodeError:
                LTC.netlist.insert(-2, instruction)
                LTC.write_netlist()
                rawfile, logfile = LTC.run()
                LTC.reset_netlist()
                LTR = LTSpiceRawRead(rawfile)
        os.remove(logfile)
    else:
        print('SIMULATION FAILED')
        with open(logfile, 'r') as f:
            [print(line) for line in f.readlines()]
        os.remove(logfile)
        return None
    return LTR

## Initializing SPICE Parameters

In [6]:
circuit_name = 'FastinoAmplifier_V1.0.cir' ## Zotino vs FastinoAmplifier vs minimalfiltering

circuit_directory_path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/lib_circuits/'
LTC = LTCommander(circuit_directory_path + circuit_name)
endsim = T0
st = max(dt/20, 1e-9) ## maximum time step of the SPICE transient sim. no lower than 100ns
slew_rate = 20/1e-6 ## slew rate in V/s (~20V/us)
gain_err = 0.01 ## 1%
calibration_err = 0.1 ## +- 100mV

print(dt/st, 'sim steps per dt')

20.0 sim steps per dt


## Run SPICE sim on naive voltages

In [31]:
plotting = True
addnoise = False
endbuffer = 23

# voltages = np.load(basepath + '/'.join((exp, 'STA_crosstalk_better.npy')), allow_pickle = True).tolist()
v_exp = {}
for k, v in v_DAC.items():
    endsim = dt*(len(v) + endbuffer)
    tevents, vevents = set_vinput(v, dt, st, slew_rate)
    
    LTR = run_spice_sim(endsim, st, 3)
    
    analysis_output = LTR.get_trace('V(ion)').get_wave(0)
    t_spice = LTR.get_trace('time').get_time_axis(0)
    print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')
    
    if addnoise:
        v_exp[k] = interp1d(t_spice, np.array(analysis_output)*(1+random.choice([-gain_err, gain_err]))
                         + random.choice([-calibration_err, calibration_err]))
    else:
        v_exp[k] = interp1d(t_spice, analysis_output)
    
    if plotting and k in ['N8', 'N9', 'S12', 'S13']:
        plt.figure(k, (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()
        
        plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input') ## fastino
#         plt.plot([t*1E6 for t in tevents], vevents, label='input') ## zotino
        plt.plot(t_spice*1E6, analysis_output, label='output')
        t_view = np.linspace(0, T0, 5000)
        plt.plot(t_view*1E6, v_ideal[k].__call__(t_view), 'k--', label='ideal')
        plt.legend(loc='best')
        
        plt.tight_layout()
plt.show()

Fri Jun 25 09:27:21 2021 : Starting simulation 1
Fri Jun 25 09:27:22 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S13	[8.         8.         8.         ... 3.64435765 1.20747873 1.08085979]	0.000402976 

Fri Jun 25 09:27:22 2021 : Starting simulation 2
Fri Jun 25 09:27:25 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N8	[1.09185489 1.09185489 1.09185489 ... 0.58705536 0.59435664 0.62865455]	0.0004030369653942924 

Fri Jun 25 09:27:25 2021 : Starting simulation 3
Fri Jun 25 09:27:27 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S12	[0.62783266 0.62783266 0.62783266 ... 0.5978911  1.08055874 1.09130176]	0.0004031077536617519 

Fri Jun 25 09:27:27 2021 : Starting simulation 4
Fri Jun 25 09:27:29 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N9	[1.1012081  1.1012081  1.1012081  ... 3.60799625 7.46571938 7.94702815]	0.00040298853631305463 



## Run SPICE sim on predistorted voltages

In [ ]:
t_view_fft, dxi = np.linspace(t_cost[0], t_cost[-1], 2**14, retstep=True)
for k, v in v_ideal_tweak.items():
    waveform = v.__call__(t_view_fft)
    waveform_fft = np.fft.rfft(waveform, 2**14)
    waveform_w = np.fft.rfftfreq(2**14, dxi)
    plt.semilogx(waveform_w, 20*np.log(np.abs(waveform_fft)/np.abs(waveform_fft[1])), marker='.', label=k)
plt.legend()
plt.title('dt=800ns waveforms, spectral content')
plt.xlabel('frequency Hz')
plt.ylabel('dB')

In [42]:
plotting = True
addnoise = False
crosstalk = False
endbuffer = 23

# newvoltages = np.load(basepath + '/'.join((exp, 'predistorted_long.npy')), allow_pickle = True).tolist()
v_exp={}
for k, v in newvoltages.items(): ## loop through each electrode
    if addnoise:
        v = np.array(v)*(1+random.choice([-gain_err, gain_err])) + random.choice([-calibration_err, calibration_err])

    endsim=dt*(len(v) + endbuffer) ## simulate a little longer so the voltages can reach their steady state
    tevents, vevents = set_vinput(v, dt, st, slew_rate)

    LTR = run_spice_sim(endsim, st, 3)
    analysis_output = LTR.get_trace("V(ion)").get_wave(0)
    t_spice = LTR.get_trace('time').get_time_axis(0)
    print('\t'.join((k, str(v), str(t_spice[-1]))), '\n')

    v_exp[k] = interp1d(t_spice, analysis_output)


    if plotting:
        plt.figure(k+'predistorted', (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()

        plt.plot([t*1E6 for t in tevents], [16*(v-2.5) for v in vevents], label='input')
        plt.plot(t_spice*1e6, analysis_output, label='output')
        plt.plot(t_spice*1e6, v_ideal_tweak[k].__call__(t_spice), 'k--', label='ideal')
        plt.legend(loc='best')

        plt.tight_layout()
plt.show()

v_of_t_crosstalk = {}
if crosstalk:
    t_eval = np.linspace(0, endsim-0.5e-6, 1000)
    varray = np.array([v.__call__(t_eval) for k, v in v_exp.items()])
    for i, v in enumerate(v_exp.values()):
        v_of_t_crosstalk[list(v_exp.keys())[i]] = interp1d(t_eval, varray[i]+0.05*(sum(varray[:i])+sum(varray[i+1:])))
        

Sun Jun 27 17:56:09 2021 : Starting simulation 1374
Sun Jun 27 17:56:10 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S13	[  8.           5.43065581   9.41475114  17.47346543  11.01700688
 -36.48094284   1.08085979]	1.176e-05 

Sun Jun 27 17:56:11 2021 : Starting simulation 1375
Sun Jun 27 17:56:14 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N8	[ 1.09185489 -0.69213431  2.56784373  3.22089908  1.29987816 -3.1769881
  0.62865455]	1.1758157714843718e-05 

Sun Jun 27 17:56:14 2021 : Starting simulation 1376
Sun Jun 27 17:56:16 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
S12	[ 0.62783266  1.2687709   0.56853074 -1.19548073 -0.74860916  4.17558354
  1.09130176]	1.1749544433593715e-05 

Sun Jun 27 17:56:16 2021 : Starting simulation 1377
Sun Jun 27 17:56:18 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
N9	[ 1.1012081   2.11531377  0.15396947 -9.45690922 -0.16744691 39.99999645
  7.94702815]	1.176080271205

# ITVG Initialization

In [9]:
#### MaxBeta Initialization
reload_vg = False
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

electrode_ordering = [
    'S21', 'N1', 'S20', 'N2', 'S19', 'N3', 'S14',
    'N4', 'S13', 'N8', 'S12', 'N9', 'S11', 'N10',
    'S10', 'N11', 'S9', 'N12', 'S8', 'N13', 'S4',
    'N14', 'S3', 'N19', 'S2', 'N20', 'S1', 'N21',
    'S25', 'S24', 'S23', 'S22'
]

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'MaxBeta_Production_Potential'

electrode_grouping = [[i+1] for i in range(len(electrode_ordering))]
# electrode_grouping = [[i+1] for i in range(len(electrode_ordering)) if i not in [18,19,22,23]]
# electrode_grouping.extend([[19,20],[23,24]])
print(electrode_grouping)

d_cons = []

start_time = time.time()
if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping=electrode_grouping, fit_ranges=[5e-6, 45e-6, 5e-6], rf_electrodes=[33], m=Ca, rf_omega=2.*np.pi*45.937e6, order=4, f_cons=d_cons, v_rf=[72])
print("Imported grid files in {}".format(time.time() - start_time))


[[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32]]
Imported grid files in 0.00010585784912109375


In [ ]:
reload_vg=True
amu = ct.m_u
e = ct.e
Ca = 40*amu
Sr = 88*amu

_electrode_ordering_old = [
    "S6", "S7", "S8", "S9", "S10", "S11", "S12", "S13", "S14",
    "S24", "S25",
    "N6", "N7", "N8", "N9", "N10", "N11", "N12", "N13", "N14",
]

def num_to_electrode_name(num):
    return _electrode_ordering[num-1]

## WHICH ONE SHOULD I USE??
# artiq_electrode_ordering = [12, 13, 14, 15, 16, 17, 18, 19, 20, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
artiq_electrode_ordering = [i+1 for i in range(20)]

path = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/GridFiles'
name = 'AngleTrap_Skinny_Potential_reduced'
name2 = 'MaxBeta_Potential'

electrode_grouping = [[x] for x in artiq_electrode_ordering] ## Lincoln Standard
electrode_grouping2 = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11]] ## Max Beta
d_cons = [('x',0), ('y',0), ('z',0)]
d_cons = []

start_time = time.time()

if reload_vg:
    vg = VoltageGeneration.VoltageGeneration(path, name, electrode_grouping, fit_ranges=[5e-6, 10e-6, 5e-6], rf_electrodes=[21], m=Sr, rf_omega=2.*np.pi*49.2e6, order=4, f_cons=d_cons, v_rf=[50])
#     vg2 = VoltageGeneration.VoltageGeneration(path, name2, electrode_grouping2, rf_electrodes=[12], m=Ca, rf_omega=2.*np.pi*43e6, order=4, f_cons=d_cons, v_rf=[70])

print("Imported grid files in {}".format(time.time() - start_time))

## Calculate Potential

In [ ]:
#### Add in the fixed electrodes
for k in electrode_ordering:
    if k in ['S22','S23','S24','S25']:
        v_exp[k] = interp1d([0, 1], [0, 0])
    elif k not in ['N9','N8','S12','S13']:
        v_exp[k] = interp1d([0, 1], [8, 8])


In [ ]:
endITVG = endsim - 0.1e-6 # we can't find the potential for voltages that don't exist! hence the minus 0.1us
volt_gen = vg
N_final = int(endITVG/100e-9)+1 # set dt (time resolution)
# N_final = len(d['S21'])
t_exp, dx = np.linspace(0, (endITVG), N_final, retstep=True)
# t_exp = np.unique(sorted(np.concatenate((t_all, t_exp))))
print(len(t_exp), 'calls per iteration \t', endITVG, 'us')

## DON'T FORGET TO CHOOSE PROPER VOLTAGES

bigarray = np.array([v_exp[k].__call__(t_exp) for k in electrode_ordering]).T
# bigarray = np.array([v_DAC[k] for k in electrode_ordering]).T
# bigarray = np.array([d[k] for k in electrode_ordering]).T
el_config = []
for row in bigarray:
    el_configi = [[[i+1], v] for i, v in enumerate(row)]
    el_configi.extend([[[el], 1] for el in volt_gen.rf_electrodes])
    el_config.append(el_configi)
    

In [ ]:
down = 1
R = [[0.0, 0, 50]]
y_span_um = (74, 373)
n_y_points = int(1e-6*(y_span_um[1]-y_span_um[0])/1000e-9)+1 # set dy (space resolution)

y_points, dy = np.linspace(y_span_um[0], y_span_um[1], n_y_points, retstep=True)
print(y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])
xaxis = y_points
yaxis = y_pot
adaptive_yaxis = True
q_min = [y_points[np.where(np.array(y) == min(np.array(y)))][0] for y in y_pot]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

X = y_points[::1]
Y = t_exp*1e6
X, Y = np.meshgrid(X, Y)
Z = y_pot[:,::1]

fig = plt.figure('3d')
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False, rstride=3, cstride=3)
ax.set_xlabel('\n position (um)', linespacing=3)
ax.set_ylabel('\n time (us)', linespacing=3)
ax.set_zlabel('\n potential (V)', linespacing=3)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

## Forward ITVG

In [ ]:
q_firstguess = q_min # initial guess for transport
# q_firstguess = np.full(len(t_exp), q0) # initial guess for splitting
num_guess = 2

q = []
coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    if not n:
        qg = q_firstguess
    else:
        qg = q_guess
        
    expansion_point = [[[0., q, 50.]] for q in qg]
    
    for i in range(len(t_exp)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        q.append(qg[i] + shift)
        print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t_exp[i]*1E6, qg[i], get_freq_from_alpha(coeffs[6], Ca), shift))))
    q_guess = q[n*len(t_exp):(n+1)*len(t_exp)]

coeff_final = np.array(coeff[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
q_exp = np.array(q[len(t_exp)*(num_guess-1):len(t_exp)*num_guess])
a_exp = coeff_final.T[6]

In [ ]:
const_map = volt_gen.function_map
x_exp = coeff_final.T[const_map['x']]
y_exp = coeff_final.T[const_map['y']]
z_exp = coeff_final.T[const_map['z']]
c_exp = coeff_final.T[const_map['yyy']]
d_exp = coeff_final.T[const_map['yyyy']]
const_exp = coeff_final.T[const_map['const']]
# plt.plot(t_exp, x_exp)
# plt.plot(t_exp, y_exp)
# plt.plot(t_exp, z_exp)

plt.plot(t_exp, get_freq_from_alpha(a_exp, Ca))
# q_exp = np.repeat(q0, len(a_exp))


In [ ]:
#### [OLD CALL]
#### Moving Harmonic Oscillator
num_guess = 1
endITVG = 20e-6
volt_gen = vg
N_final = int(endITVG*1e6*5)+1
t_anal = np.linspace(0, (endITVG), N_final)
print(N_final, 'calls per iteration')

## DON'T FORGET TO CHANGE THE FIRST GUESS
q_firstguess = np.array([qtrig(t, (1e-6, 21e-6), qspan=(120, 240), f=1.2e6) for t in t_anal])

## DON'T FORGET TO CHOOSE PROPER VOLTAGES
bigarray = np.array([v.__call__(t_anal) for k, v in v_of_t_predist.items()]).T

el_config = []
for row in bigarray:
    el_config.append([[[i+1], v] for i, v in enumerate(row)])

q = []
coeff = []
for n in range(num_guess):
    print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    if not n:
        qg = q_firstguess
    else:
        qg = q_guess
        
    expansion_point = [[[0., i, 50.]] for i in qg]
        
    for i in range(len(t_anal)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        q.append(qg[i] + shift)
        print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t_anal[i]*1E6, qg[i], get_freq_from_alpha(coeffs[6], Sr), shift))))
    q_guess = q[n*N_final:(n+1)*N_final]

coeff_final = np.array(coeff[N_final*(num_guess-1):N_final*num_guess])
q_final = np.array(q[N_final*(num_guess-1):N_final*num_guess])
a_final = coeff_final.T[6]

In [ ]:
# plt.plot(t_anal*1e6, q_firstguess, 'k')
# plt.plot(t_anal*1e6, q_final)

delq = q_firstguess - q_final
plt.plot(t_anal*1e6, delq)

# Predistortion

## Deconvolution

### Electrode Response

In [7]:
#### impulse response of circuit  
periods = 1
endbuffer = (2*periods-1)*Ntot
step = True
if step:
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip([0, 1E-12], [2.5, 1/16+2.5])]
else:
    with open(circuit_directory_path + 'input.txt', 'w+') as f:
        [f.write('\t'.join((str(t), str(v))) + '\n') for t, v in zip([0, 1E-12, 100E-9], [2.5, 1/16+2.5, 2.5])]
LTR = run_spice_sim(T0 + endbuffer*dt, st, 3)
analysis_output = LTR.get_trace("V(ion)").get_wave(0)
t_spice = LTR.get_trace('time').get_time_axis(0)
ended = float(t_spice[-1])

#### FFT of impulse response
N = 2**16
M = int(100*N*ended)
t_fft, dx = np.linspace(0, ended, M, retstep=True)
print(M, N/M, 1/dx)
if step:
    h_step_t = InterpolatedUnivariateSpline(t_spice, analysis_output).__call__(t_fft)
    h_t = np.gradient(h_step_t, dx)*dx
else:
    h_t = InterpolatedUnivariateSpline(t_spice, analysis_output).__call__(t_fft)
h_fft = np.fft.rfft(h_t, N)
h_w = np.fft.rfftfreq(N, dx)
h_mag = np.abs(h_fft)
h_phi = np.unwrap(np.angle(h_fft))

h_phase = -h_phi/np.radians(h_w)
h_group = -np.gradient(h_phi, np.radians(h_w[1]-h_w[0]))

plt.figure('impulse response')
plt.plot(t_fft*1e6, h_t)
plt.ylabel('voltage (V)')
plt.xlabel('time (us)')
plt.figure('impulse mag')
plt.loglog(h_w, h_mag, marker='.')
plt.xlabel('frequency Hz')
plt.ylabel('magnitude')
# plt.figure('impulse phase')
# plt.semilogx(h_w, h_phase, ls='--', label='phase delay', color='tab:orange')
# plt.semilogx(h_w, h_group, lw=3, label='group delay', color='tab:orange')
# plt.xlabel('frequency Hz')
# plt.ylabel('magnitude')
plt.legend(loc='best', shadow=1)


Tue Jul 27 11:39:48 2021 : Starting simulation 1


/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
No handles with labels found to put in legend.


Tue Jul 27 11:39:49 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
3866 16.951888256595964 6550847.4576271195


In [ ]:
v_ideal_deconv

### Weiner Deconvolution

In [11]:
x_n = {}
SNR = np.array([1e6 if w < 120e3 else 500 if w < 360e3 else 10 for w in h_w])
for k, v in v_ideal_deconv.items():
    if k not in x_n.keys():
#     if k == 'S12':
        #### FFT of ideal v(t)
        v_t = v.__call__(t_fft%(2*T0))
        vavg = np.average(v_t)
        v_t -= vavg
        v_fft = np.fft.rfft(v_t, N)
        v_t += vavg

        #### Wiener deconvolution
        x_fft = v_fft/(h_fft*(1+1/(np.abs(h_fft)**2*SNR)))
        x_t = np.fft.irfft(x_fft, N)
        x_t = x_t[:M] + vavg

        #### SPICE sim of deconvolved input
        tevents, vevents = set_vinput(x_t, dx, st, slew_rate)
        LTR = run_spice_sim(t_fft[-1], st, 3)
        analysis_output = LTR.get_trace("V(ion)").get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)

        plt.figure(k, (19, 9))
        plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, x_t[:int(len(t_fft)/periods)], 'r-.')
        plt.plot(t_fft[:int(len(t_fft)/periods)]*1e6, v_t[:int(len(t_fft)/periods)], 'k--')
        plt.plot(t_spice[:int(len(t_spice)/periods)]*1e6, analysis_output[:int(len(t_spice)/periods)])
        plt.title(k)
        plt.tight_layout()

        #### Sample deconvolved signal
        x_n[k] = interp1d(t_fft, x_t).__call__([(n+0.5)*dt for n in range(Ntot+endbuffer)])

Tue Jul 27 11:41:08 2021 : Starting simulation 2
Tue Jul 27 11:41:09 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:09 2021 : Starting simulation 3
Tue Jul 27 11:41:09 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:09 2021 : Starting simulation 4
Tue Jul 27 11:41:10 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:10 2021 : Starting simulation 5
Tue Jul 27 11:41:11 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:11 2021 : Starting simulation 6
Tue Jul 27 11:41:12 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:12 2021 : Starting simulation 7
Tue Jul 27 11:41:12 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:13 2021 : Starting simulation 8
Tue Jul 27 11:41:13 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:41:13 2021 : Starting simulation 9

/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Tue Jul 27 11:42:05 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:42:05 2021 : Starting simulation 24


/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Tue Jul 27 11:42:05 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:42:05 2021 : Starting simulation 25


...

Tue Jul 27 11:42:06 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access
Tue Jul 27 11:42:06 2021 : Starting simulation 26


/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Tue Jul 27 11:42:07 2021: Simulation Successful. Time elapsed 00:00:00:

Normal access


/Users/lukeqi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


KeyboardInterrupt: 

## Optimization

In [ ]:
plt.plot(t_cost, np.gradient(v_ideal_tweak['N9'].__call__(t_cost))*20)
plt.plot(t_cost[:-2], np.gradient(v_exp['N9'].__call__(t_cost[:-2]))*20)

In [ ]:
plt.plot(t_cost, v_ideal_tweak['S13'].__call__(t_cost))

In [9]:
def float_to_16bit(n):
    ''' n is an array of floats
    DAC resolution is 16-bit from -10V to 10V '''
    bit16 = 2**16-1
    if type(n) == 'float':
        return int((n+10)/20*bit16)/bit16*20-10
    return [int((i+10)/20*bit16)/bit16*20-10 for i in n]

prevx = []
prev_elv = []
def spice_sim_cost(x, *args):
    ''' x is a list of input voltages of length N
        returns cost function between SPICE output and idealized potential q(t) '''    
    global prevx
    global prev_elv
    global electrode_ordering
    t = args[0]
    q = args[1]
    endsim = t[-1]+0.5*dt
    x = x.reshape(-1, N0)
    beginv = np.repeat(args[3][0], args[2][0], axis=1)
    endv = np.repeat(args[3][-1], args[2][-1], axis=1)
    if args[3]:
        x = np.concatenate((beginv, x, endv), axis=1)
    
    #### Spice simulation of all the electrodes
    v_output = []
    electrodev = []
    for k in electrode_ordering:
        if k == 'S13':
            if list(x[0]) not in prevx:
                set_vinput(x[0], dt, st, slew_rate)

                LTR = run_spice_sim(endsim, st, 25)
                analysis_output = LTR.get_trace('V(ion)').get_wave(0)
                t_spice = LTR.get_trace('time').get_time_axis(0)
                v_output.append(interp1d(t_spice, analysis_output).__call__(t))
                electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
            else:
                v_output.append(prev_elv[0])
                electrodev.append(prev_elv[0])
        elif k == 'N8':
            if list(x[1]) not in prevx:
                set_vinput(x[1], dt, st, slew_rate)

                LTR = run_spice_sim(endsim, st, 25)
                analysis_output = LTR.get_trace('V(ion)').get_wave(0)
                t_spice = LTR.get_trace('time').get_time_axis(0)
                v_output.append(interp1d(t_spice, analysis_output).__call__(t))
                electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
            else:
                v_output.append(prev_elv[1])
                electrodev.append(prev_elv[1])
        elif k == 'S12':
            if list(x[2]) not in prevx:
                set_vinput(x[2], dt, st, slew_rate)

                LTR = run_spice_sim(endsim, st, 25)
                analysis_output = LTR.get_trace('V(ion)').get_wave(0)
                t_spice = LTR.get_trace('time').get_time_axis(0)
                v_output.append(interp1d(t_spice, analysis_output).__call__(t))
                electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
            else:
                v_output.append(prev_elv[2])
                electrodev.append(prev_elv[2])
        elif k == 'N9':
            if list(x[3]) not in prevx:
                set_vinput(x[3], dt, st, slew_rate)

                LTR = run_spice_sim(endsim, st, 25)
                analysis_output = LTR.get_trace('V(ion)').get_wave(0)
                t_spice = LTR.get_trace('time').get_time_axis(0)
                v_output.append(interp1d(t_spice, analysis_output).__call__(t))
                electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
            else:
                v_output.append(prev_elv[3])
                electrodev.append(prev_elv[3])
        elif k in ['S22','S23','S24','S25']:
            v_output.append(np.full(len(t), 0))
        else:
            v_output.append(np.full(len(t), 8))
            
#     for i, el_v in enumerate(x):
#         if list(el_v) not in prevx:
#             set_vinput(el_v, dt, st, slew_rate)

#             LTR = run_spice_sim(endsim, st, 25)
#             analysis_output = LTR.get_trace('V(ion)').get_wave(0)
#             t_spice = LTR.get_trace('time').get_time_axis(0)
#             electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
#         else:
#             electrodev.append(prev_elv[i])
    
    #### Forward ITVG
    volt_gen = args[4]
    el_config = []
    for row in np.array(v_output).T:
        el_config.append([[[i+1], v] for i, v in enumerate(row)])

    delta = []
    coeff_final = []
#     print('\t'.join(('iter', 't [us]', 'R [um]', 'f [Hz]', 'shift [um]')))
    expansion_point = [[[0., i, 50.]] for i in q]

    for i in range(len(t)):
        coeffs = volt_gen.compute_total_potential_axes(expansion_point[i], el_config[i], printing=False)
        coeff_final.append(coeffs)
        shift = -coeffs[1]/(2*coeffs[6])
        delta.append(shift)
#         print('\t'.join((str(i), '{:.2f}\t{:.2f}\t{:.0f}\t{:.3f}'.format(t[i]*1E6, q[i], get_freq_from_alpha(coeffs[6], Sr), shift))))
    coeff_final = np.array(coeff_final)
    a_final = coeff_final.T[6]
    
    #### cost function
    cost = sum(np.array(delta)**2)
    a0 = get_alpha_from_freq(4e6, m_=Ca)
    cost += sum((a0/1e12 - a_final)**2)
    print('cost: ', cost)

    prevx = list([list(r) for r in np.copy(x)])
    prev_elv = electrodev
    return cost

def spice_sim_cost2(x, *args):
    ''' x is a list of input voltages of length N
        returns cost function between SPICE output and idealized v(t) '''    
    global prevx
    global prev_elv
    global v_ideal_tweak
    t = args[0]
    q = args[1]
    endsim = t[-1]+0.5*dt
    x = x.reshape(-1, N0)
    beginv = np.repeat(args[3][0], args[2][0], axis=1)
    endv = np.repeat(args[3][-1], args[2][-1], axis=1)
    if args[3]:
        x = np.concatenate((beginv, x, endv), axis=1)
    
    #### Spice simulation of all the electrodes
    electrodev = []
    for i, el_v in enumerate(x):
        if list(el_v) not in prevx:
            set_vinput(el_v, dt, st, slew_rate)

            LTR = run_spice_sim(endsim, st, 25)
            analysis_output = LTR.get_trace('V(ion)').get_wave(0)
            t_spice = LTR.get_trace('time').get_time_axis(0)
            electrodev.append(interp1d(t_spice, analysis_output).__call__(t))
        else:
            electrodev.append(prev_elv[i])

    #### cost function
    cost = 0 
    for i, (k, v) in enumerate(v_ideal_tweak.items()):
        cost += sum((np.array(electrodev[i]) - v.__call__(t))**2)
        cost += sum((np.gradient(np.array(electrodev[i]))*20 - np.gradient(v.__call__(t))*20)**2)
    print('cost: ', cost, '\t x:', x)

    prevx = list([list(r) for r in np.copy(x)])
    prev_elv = electrodev
    return cost

In [ ]:
plt.plot(t_tweak%(2*T0), v_ideal_deconv['N9'].__call__(t_tweak%(2*T0)))

In [12]:
# ti, tf = (288, 294) ## multiples of dt
# ti, tf = (287, 296)
# ti, tf = (473, 485)
# ti, tf = (991, 1008)
ti, tf = (987, 1005)
begin, end = (1, 13) ## N0 = (tf - ti + 1) - (begin + end)
t_cost = np.linspace(0, 30e-6, 500)

vguess = {}
v_ideal_tweak = {}
t_tweak = dt*np.linspace(ti, tf, 100)
for k, v in v_ideal_deconv.items():
    vguess[k] = np.concatenate((begin*[v.__call__(ti*dt)], x_n[k][ti+begin:tf-end+1], end*[v.__call__(tf*dt)]))
    v_ideal_tweak[k] = interp1d(np.append(t_tweak-ti*dt, 1), np.append(v.__call__(t_tweak%(2*T0)), v.__call__(tf*dt%(2*T0))))

# newvoltages = {}
run_minimize = False
if run_minimize:
    q_ideal = np.array([qsta2(t, (begin*1e-6, (N0+begin)*1e-6), qspan=(186.5, 231.5), f=4e6*5e-6) for t in t_cost])
    bufferv = np.array([v.__call__([0, 1]) for k, v in v_ideal_tweak.items()])
    vguessi = np.array([v[begin:-end] for k, v in vguess.items()]).reshape(-1) ## flatten electrode commands to 1D array
#     x_ideal = minimize(spice_sim_cost, vguessi, args=(t_cost, q_ideal, (begin, end), (bufferv[:, 0:1], bufferv[:, -1:]), vg), bounds=[(-40,40)]*(len(vguessi)), 
#                        method='SLSQP', options={'disp':True, 'ftol':1e-7, 'eps': 0.1, 'maxiter':100})
    x_ideal = minimize(spice_sim_cost2, vguessi, args=(t_cost, q_ideal, (begin, end), (bufferv[:, 0:1], bufferv[:, -1:])), bounds=[(-40,40)]*(len(vguessi)), 
                       method='SLSQP', options={'disp':True, 'ftol':1e-7, 'eps': 0.1, 'maxiter':100})
    print(x_ideal)
    newvoltages[k] = float_to_16bit(np.concatenate((vguess[k][:begin], x_ideal.x, vguess[k][-end:])))
else:
    for k, v in v_ideal_tweak.items():
        tevents, vevents = set_vinput(vguess[k], dt, st, slew_rate)
        LTR = run_spice_sim(t_cost[-1], st, 3)
        analysis_output = LTR.get_trace("V(ion)").get_wave(0)
        t_spice = LTR.get_trace('time').get_time_axis(0)

        plt.figure(k, (19, 9))
        plt.title('Electrode %s' % (k, ))
        plt.xlabel('Time [us]')
        plt.ylabel('Voltage [V]')
        plt.grid()
        plt.plot([t*1e6 for t in tevents], [16*(v-2.5) for v in vevents], label='input')
        plt.plot(t_spice*1e6, analysis_output, label='output')
        plt.plot(t_spice*1e6, v_ideal_tweak[k].__call__(t_spice), 'k--', label='ideal')
        plt.legend(loc='best')

#         newvoltages[k] = vguess[k]

# np.save(basepath + '/'.join((exp, 'voltages_predist.npy')), newvoltages)

ValueError: A value in x_new is above the interpolation range.

In [ ]:
print(prevx)
print(np.array(prevx).T[1:-1].T)

## Fine Adjustments

In [39]:
#### make finer adjustments to improve optimization
allvoltages = np.concatenate([bufferv[:,:-1], x_ideal.x.reshape(-1, N0), bufferv[:,-1:]], axis=1)
# print(allvoltages)
# allvoltages = np.concatenate([bufferv[:,:-1], np.array(prevx).T[1:-1].T.reshape(-1, N0), bufferv[:,-1:]], axis=1)
newvoltages = {k: allvoltages[i] for i, k in enumerate(list(d.keys()))}
print(newvoltages)

# exp = ''
# np.save(basepath + '/'.join((exp, 'predistorted_long.npy')), newvoltages)


{'S13': array([  8.        ,   5.43065581,   9.41475114,  17.47346543,
        11.01700688, -36.48094284,   1.08085979]), 'N8': array([ 1.09185489, -0.69213431,  2.56784373,  3.22089908,  1.29987816,
       -3.1769881 ,  0.62865455]), 'S12': array([ 0.62783266,  1.2687709 ,  0.56853074, -1.19548073, -0.74860916,
        4.17558354,  1.09130176]), 'N9': array([ 1.1012081 ,  2.11531377,  0.15396947, -9.45690922, -0.16744691,
       39.99999645,  7.94702815])}


In [ ]:
t_view=np.linspace(0E-6, 34e-6, 1000)
plt.figure(100)
for k, v in v_ideal_tweak.items():
    plt.plot(t_view*1e6, v.__call__(t_view), label=k)
plt.legend(loc='best', prop={'size': 12})
plt.ylabel('voltage (V)')

# plt.figure(101)
# for k, v in v_of_t.items():
#     plt.plot(t_view*1e6, v.__call__(t_view), label=k)
# plt.legend(loc='best', prop={'size': 12})
# plt.ylabel('Electrode Voltage (V)')
# plt.xlabel('Step')    
    
plt.figure(102)
for k, v in v_of_t_predist.items():
    plt.plot(t_view*1e6, v.__call__(t_view))

In [40]:
v_DAC_final = {}
for k in electrode_ordering:
    if k in ['S13', 'N8', 'S12', 'N9']:
        v_DAC_final[k] = newvoltages[k]
    elif k in ['S22','S23','S24','S25']:
        v_DAC_final[k] = [0]*len(newvoltages['N9'])
    else:
        v_DAC_final[k] = [8]*len(newvoltages['N9'])

print(v_DAC_final)

{'S21': [8, 8, 8, 8, 8, 8, 8], 'N1': [8, 8, 8, 8, 8, 8, 8], 'S20': [8, 8, 8, 8, 8, 8, 8], 'N2': [8, 8, 8, 8, 8, 8, 8], 'S19': [8, 8, 8, 8, 8, 8, 8], 'N3': [8, 8, 8, 8, 8, 8, 8], 'S14': [8, 8, 8, 8, 8, 8, 8], 'N4': [8, 8, 8, 8, 8, 8, 8], 'S13': array([  8.        ,   5.43065581,   9.41475114,  17.47346543,
        11.01700688, -36.48094284,   1.08085979]), 'N8': array([ 1.09185489, -0.69213431,  2.56784373,  3.22089908,  1.29987816,
       -3.1769881 ,  0.62865455]), 'S12': array([ 0.62783266,  1.2687709 ,  0.56853074, -1.19548073, -0.74860916,
        4.17558354,  1.09130176]), 'N9': array([ 1.1012081 ,  2.11531377,  0.15396947, -9.45690922, -0.16744691,
       39.99999645,  7.94702815]), 'S11': [8, 8, 8, 8, 8, 8, 8], 'N10': [8, 8, 8, 8, 8, 8, 8], 'S10': [8, 8, 8, 8, 8, 8, 8], 'N11': [8, 8, 8, 8, 8, 8, 8], 'S9': [8, 8, 8, 8, 8, 8, 8], 'N12': [8, 8, 8, 8, 8, 8, 8], 'S8': [8, 8, 8, 8, 8, 8, 8], 'N13': [8, 8, 8, 8, 8, 8, 8], 'S4': [8, 8, 8, 8, 8, 8, 8], 'N14': [8, 8, 8, 8, 8, 8, 8], 'S3':

In [41]:
basepath = '/Users/lukeqi/Desktop/School/MIT/UROP/SuperUROP/IonMotion/'
filepath = 'MB_fasttransport_4MHz_dt=392ns_N=5_R8=13k.npy'

newdat = []
for i in range(len(v_DAC_final['N10'])):
    newdat.append({k: v[i] for k, v in v_DAC_final.items()})
newdat = np.array(newdat)

np.save(os.path.join(basepath, filepath), newdat)
print(newdat)

[{'S21': 8, 'N1': 8, 'S20': 8, 'N2': 8, 'S19': 8, 'N3': 8, 'S14': 8, 'N4': 8, 'S13': 8.0, 'N8': 1.0918548909329517, 'S12': 0.6278326635904425, 'N9': 1.101208104781884, 'S11': 8, 'N10': 8, 'S10': 8, 'N11': 8, 'S9': 8, 'N12': 8, 'S8': 8, 'N13': 8, 'S4': 8, 'N14': 8, 'S3': 8, 'N19': 8, 'S2': 8, 'N20': 8, 'S1': 8, 'N21': 8, 'S25': 0, 'S24': 0, 'S23': 0, 'S22': 0}
 {'S21': 8, 'N1': 8, 'S20': 8, 'N2': 8, 'S19': 8, 'N3': 8, 'S14': 8, 'N4': 8, 'S13': 5.430655812433442, 'N8': -0.6921343113375692, 'S12': 1.2687709009537462, 'N9': 2.1153137668345847, 'S11': 8, 'N10': 8, 'S10': 8, 'N11': 8, 'S9': 8, 'N12': 8, 'S8': 8, 'N13': 8, 'S4': 8, 'N14': 8, 'S3': 8, 'N19': 8, 'S2': 8, 'N20': 8, 'S1': 8, 'N21': 8, 'S25': 0, 'S24': 0, 'S23': 0, 'S22': 0}
 {'S21': 8, 'N1': 8, 'S20': 8, 'N2': 8, 'S19': 8, 'N3': 8, 'S14': 8, 'N4': 8, 'S13': 9.414751137046444, 'N8': 2.567843732555237, 'S12': 0.5685307448284299, 'N9': 0.15396947094326904, 'S11': 8, 'N10': 8, 'S10': 8, 'N11': 8, 'S9': 8, 'N12': 8, 'S8': 8, 'N13': 8,

# Ion Simulation

In [ ]:
def heat_sim(tff, q_protocol, t_eval, ms=1E-8, showplot=False, a=None, b=None, g=None, c=None, deg=2):
    ''' simulates ion motion and returns heat gained
        parameters: final integration time, motion protocol, time window for heat calculation 
        all length units must be in um'''
    if a is not None:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(a*1e12, a[-1]*1e12), k=deg)
    else:
        alpha = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [get_alpha_from_freq(omega_0/(2*pi))]*(len(t_anal)+1), k=deg)
    if b is not None:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(b*1e24, b[-1]*1e24), k=deg)
    else:
        beta = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    if g is not None:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(g*1e6, g[-1]*1e6), k=deg)
    else:
        gamma = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    if c is not None:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.append(c*1e18, c[-1]*1e18), k=deg)
    else:
        cubic = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), [0]*(len(t_anal)+1), k=deg)
    qi = InterpolatedUnivariateSpline(np.concatenate((t_anal,[tff*1.1])), np.concatenate((q_protocol/1e6, [q_protocol[-1]/1e6])), k=deg)
    
    def fn(t, v):
        x = v[:Nion]
        dxdt = v[Nion:]
        dist = get_dist_matrix(x)
        
        eom = [] ## equation of motion
        for i in range(Nion): ## iterate over all the ions
            eom.append(-q_ions[i]/m[i]*p_derivative(phi, point=[t, x[i]-qi.__call__(t), alpha.__call__(t), beta.__call__(t), m[i], gamma.__call__(t), cubic.__call__(t)]) +
                       k0*q_ions[i]/m[i]*sum([q_ions[j]*abs(dist[i][j])/(dist[i][j]**3) for j in range(Nion) if j!=i]))
        
        return np.concatenate([dxdt, eom])
        
    outsol = solve_ivp(fn, (0, tff), np.concatenate([x0, x0dot]), method='RK45', dense_output=True, t_eval=t_eval, max_step=ms)    
    h = get_heat(outsol.y[0]-qi.__call__(t_eval), 2*pi*get_freq_from_alpha(alpha.__call__(tff)/1e12, m[0]), m[0])
    
    if showplot:
        t_view = np.linspace(0, tff, int(1E9*tff))
        q = qi.__call__(t_view)
        x = outsol.sol(t_view)[0]
    #     dx=t_view[1]-t_view[0]
    #     dqdt=np.gradient(q,dx)
    #     d2qdt2=np.gradient(dqdt, dx)
    #     d3qdt3=np.gradient(d2qdt2, dx)

        fig, ax = plt.subplots()
        plt1, = ax.plot(t_view*1E6, q*1E6, 'k--')
        ax.set_ylabel('potential minimum position (um)', color='k')
        ax2=ax.twinx()
        plt2, = ax2.plot(t_view*1E6, (x-q)*1E9)
        ax2.set_ylabel('ion oscillation (nm)')
        plt3 = ax2.axvline(t_eval[0]*1E6, color='r', linestyle='--')
        ax2.axvline(t_eval[-1]*1E6, color='r', linestyle='--')
    #     ax.plot(t_view*1E6, dqdt)
    #     ax.plot(t_view*1E6, d2qdt2)
    #     ax.plot(t_view*1E6, d3qdt3, 'g')
        ax.set_xlabel('time (us)')
        plt.legend((plt1, plt2, plt3), ('potential minimum', 'ion motion', 'heat calculation window'), loc='best')
        
        for n in range(1, Nion):
            x = outsol.sol(t_view)[n]
            ax2.plot(t_view*1E6, (x-q)*1E9)
        
    return outsol, h, outsol.y, qi.__call__(t_eval)

def phi(t, y, a, b, m, g=0, c=0):
    ''' electric potential function [SI Units]
    y is the difference between ion position and potential minimum 
    '''
    return a*y**2 + b*y**4 + g*y + c*y**3

def p_derivative(func, ms=1e-11, var=1, point=[], order=1):
    ''' implements a partial derivative '''
    args = point[:]
    def wraps(x):
        args[var] = x
        return func(*args)
    return derivative(wraps, point[var], dx = ms, n=order)
    
def get_dist_matrix(y):
    ''' gets distance matrix given positions of N ions 
        x = Nion x 1 vector '''
    D=np.zeros((Nion, Nion))
    for i in range(Nion):
        for j in range(i+1, Nion):
            D[i][j]=y[i]-y[j]
            D[j][i]=-D[i][j]
    return D

def get_heat(yvalues, w, m_=40*ct.m_u): ## calculates <n0> (average motional state) Given by Hercul 3.2 eq 97
    maxy = np.max(yvalues)
    miny = np.min(yvalues)
    A = (maxy-miny)/2
    return A**2*m_*np.abs(w)/(2*hbar)

def find_ic(q0, a0, N, xguess, b0=0, disp=False):
    ''' given q, phi coefficients, and num ion, minimize the total energy of the system
    both q and coeff have length scale in um'''
    def total_potential_energy(x, *args):
        q0, a0, b0 = args
        dist = get_dist_matrix(x)
        U = sum([a0*q_ions[i]*(x[i]-q0)**2 + b0*q_ions[i]*(x[i]-q0)**4 + 0.5*k0*q_ions[i]*sum([q_ions[j]/abs(dist[i][j])
                                                                   for j in range(N) if j!=i]) for i in range(N)])
        return U/1.6E-19
    idealx = minimize(total_potential_energy, xguess, args=(q0, a0, b0), method='SLSQP',
                     options={'ftol': 1e-12, 'eps': 1E-12, 'maxiter':1000})
    if disp:
        print(idealx)
    return idealx.x

In [ ]:
hbar = ct.hbar
eps0 = ct.epsilon_0
k0 = 1/(4*pi*eps0)
amu = ct.m_u
elem_charge = ct.e
um = 1e-6

Nion = 1
q_ions = elem_charge*   np.full(Nion, 1)
m = amu*                np.array([88, 88])
x0dot = um*             np.full(Nion, 0)
x0guess = um*           np.array([q_final[0]])

# Ti = 2e-6
# t_anal = np.linspace(0, Ti, 1000)
# a_final = np.full(len(t_anal), get_alpha_from_freq(1.2e6, m_=88*amu)/1e12)
# q_final = np.array([qsta2(t, (0e-6, Ti), qspan=(-60, 60), f=Ti*1.06e6) for t in t_anal])
#### Transport
x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess)
print(x0*1e6)

#### Splitting
# x0 = find_ic(q_final[0]/1e6, a_final[0]*1e12, Nion, x0guess)
# print(x0*1e6)
# plt.plot(y_points, y_pot[0])

In [ ]:
heat_start = 50e-6
heat_end = 60e-6

t_eval = np.linspace(heat_start, heat_end, 4096)
outsol, n, y_eval, q_eval = heat_sim(t_eval[-1], q_final, t_eval, showplot=True, a=a_final, b=None, g=None, c=None, deg=2)
print('heat gained (quanta): ', n)

# Potential Animation

In [ ]:
#### Splitting potential shape
down = 1
R = [[0.0, 0, 50.0]]
volt_gen = vg

## 

# x_bounds = [-20.0, 20.0]
# z_bounds = [40.0, 80.0]
# pot_values = np.array([total_potential_cut(volt_gen, b, R[0], 0.0, x_bounds, z_bounds) for b in bigarray[::down]])
# r_values = np.transpose(pot_values, (1, 0, 2))[0][0]
# pot_values = np.transpose(pot_values, (1, 0, 2))[1]

##

y_span_um = 420
n_y_points = 420
y_points = np.linspace(-y_span_um, y_span_um, n_y_points)
y_test_points = np.array([np.zeros(n_y_points), y_points, np.zeros(n_y_points)]).T + R
y_pot = np.array([volt_gen.compute_potential(y_test_points, [[el[0][0], el[1]] for el in e]) for e in el_config[::down]])

## change what you're plotting
xaxis = y_points
yaxis = y_pot
adaptive_yaxis = False

%matplotlib inline
fig, ax = plt.subplots()
# ax.set_ylim(min(yaxis.flatten()), max(yaxis.flatten()))
ax.set_ylim(-3, 1)
line, = ax.plot(xaxis, yaxis[0])

def animate(i):
    if adaptive_yaxis:
#         ax.set_ylim(min(yaxis[i]), max(yaxis[i]))
        ax.set_ylim(yaxis[i][find_peaks(-yaxis[i])[0][0]], max(yaxis[i]))
    line.set_ydata(yaxis[i])  # update the data
    return line,

#Init only required for blitting to give a clean slate.
def init():
    line.set_ydata(np.ma.array(xaxis, mask=True))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(1, int(len(t_anal)/down)-1), init_func=init,
    interval=25, blit=True)
HTML(ani.to_jshtml())